In [18]:
import pandas as pd
from bs4 import BeautifulSoup
import csv

HTML PARSER

In [2]:
# Read the HTML content (assuming it's saved as 'cc.html')
with open('..\\data\\backtest_results.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse HTML with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Find the table
table = soup.find('table')

# Extract headers
headers = []
header_row = table.find('tr', bgcolor="#E5F0FC")  # Find the header row by its background color
for th in header_row.find_all('td'):
    headers.append(th.text.strip())

# Extract data rows
data_rows = []
for tr in table.find_all('tr')[2:-2]:  # Skip header rows and footer rows
    row = []
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    data_rows.append(row)

# Write to CSV
output_file = 'deals.csv'
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    
    # Write headers
    writer.writerow(headers)
    
    # Write data rows
    writer.writerows(data_rows)

print(f"Data has been successfully written to {output_file}")

Data has been successfully written to deals.csv


In [10]:
# Load backtest results
file_path = 'deals.csv'  # Update with your actual file path
data = pd.read_csv(file_path)

# Ensure datetime format if there's a timestamp column
if 'Date' in data.columns:
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Num', inplace=True)

data.drop(columns=['Swap', 'Symbol', 'Order'], inplace=True)

# Inspect the data
data.head()

,Date,Type,Direction,Volume,Price,Commission,Profit,Balance,Comment
Num,,,,,,,,,
2,2023-02-07 14:07:46,sell,in,0.28,1869.69,-0.84,0.00,9 999.16,NaN
3,2023-02-07 17:36:04,buy,out,0.28,1872.34,0.00,-74.20,9 924.96,sl 1872.32
4,2023-02-09 03:09:41,sell,in,0.27,1872.45,-0.81,0.00,9 924.15,NaN
5,2023-02-09 03:37:37,buy,out,0.27,1876.10,0.00,-98.55,9 825.60,sl 1876.10
6,2023-02-09 10:45:19,buy,in,0.15,1884.60,-0.45,0.00,9 825.15,NaN


In [ ]:
# Clean numeric columns (remove spaces and convert to float)
numeric_cols = ['Price', 'Commission', 'Profit', 'Balance']
data[numeric_cols] = data[numeric_cols].astype(str).apply(lambda x: x.str.replace(' ', '').astype(float))
# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])
data.head()

,Date,Type,Direction,Volume,Price,Commission,Profit,Balance,Comment
Num,,,,,,,,,
2,2023-02-07 14:07:46,sell,in,0.28,1869.69,-0.84,0.00,9 999.16,NaN
3,2023-02-07 17:36:04,buy,out,0.28,1872.34,0.00,-74.20,9 924.96,sl 1872.32
4,2023-02-09 03:09:41,sell,in,0.27,1872.45,-0.81,0.00,9 924.15,NaN
5,2023-02-09 03:37:37,buy,out,0.27,1876.10,0.00,-98.55,9 825.60,sl 1876.10
6,2023-02-09 10:45:19,buy,in,0.15,1884.60,-0.45,0.00,9 825.15,NaN


Pair Entry and Exit Trades

In [25]:
trades = []

# Iterate through pairs of rows (entry 'in' and exit 'out')
for i in range(0, len(data)-1, 2):
    entry = data.iloc[i]
    exit = data.iloc[i+1]
    
    if entry['Direction'] == 'in' and exit['Direction'] == 'out':
        trade = {
            'EntryDate': entry['Date'],
            'ExitDate': exit['Date'],
            'Type': entry['Type'],
            'Volume': entry['Volume'],
            'EntryPrice': entry['Price'],
            'ExitPrice': exit['Price'],
            'Profit': exit['Profit'],
            'Balance': exit['Balance'],
            'StopLoss': 'sl' in str(exit['Comment']).lower(),
            'Duration': round((exit['Date'] - entry['Date']).total_seconds() / 60, 2)  # in minutes
        }
        trades.append(trade)

trades_df = pd.DataFrame(trades)

# Clean numeric columns (remove spaces and convert to float)
numeric_cols = ['Volume', 'EntryPrice', 'ExitPrice', 'Profit', 'Balance']
trades_df[numeric_cols] = trades_df[numeric_cols].astype(str).apply(lambda x: x.str.replace(' ', '').astype(float))

trades_df.to_csv('trades.csv')

Feature Engineering